In [81]:
import pandas as pd
import time 
from copy import copy

class Operation:
    """Read or Write"""
    READ = "read"
    WRITE = "write"

    def __init__(self, _id, target, operation_type, function = None):
        self.id = _id
        #Operation.target est un pointeur vers la vraie target de l'Opération
        #Operation.local_target est un pointeur vers une des local target de la Transaction
        self.target = target 
        self.target_local = None 
        self.function = function
        self.operation_type = operation_type
        if self.operation_type == Operation.WRITE and self.function == None:
            raise Exception("A WRITE Operation was not provided a function !")
        self.has_applied = False

    def apply(self) -> bool :
        # Defin a local copy of the target object that we will modify
        if self.operation_type == Operation.READ:
            # Do nothing (simulate a read)
            return True
        elif self.operation_type == Operation.WRITE: 
            self.target_local = copy(self.target)
            success = self.target_local.apply(self)
            self.has_applied = success
            return success
        else:
            return False

    def write_changes(self) -> bool:
        if self.operation_type == Operation.READ:
            # No change to commit.
            return True
        elif self.operation_type == Operation.WRITE:
            # Write local changes to global target
            self.target = self.target_local
            return True # success
        else: # unknown operation type 
            False


In [83]:
def list_intersect(l1, l2):
    intersect = []
    for item in l1:
        if item in l2:
            interesect.append(l2)
    return intersect

class Transaction:
    phase_READ = 'read'
    phase_VALIDATE = "validate"
    phase_WRITE = "write"
    phase_FINISHED = "finished"

    def __init__(self, operation_list):
        self.operation_list = operation_list
        self.operation_iter = 0
        # Current phase. None means that we didn't start. 
        self.phase = None
        # Timestamps of different phases
        self.ts_start_read = None
        self.ts_start_validate = None
        self.ts_start_write = None
        self.ts_finished = None
        # Create local copies of target objects
        self.init_target_local()
        # Define read and write sets
        self.init_read_set()
        self.init_write_set()

    def init_read_set(self):
        self.read_set = []
        for operation in self.operation_list:
            if operation.operation_type == Operation.READ and operation.target not in self.read_set:
                self.read_set.append(operation.target)

    def init_write_set(self):
        self.write_set = []
        for operation in self.operation_list:
            if operation.operation_type == Operation.WRITE and operation.target not in self.write_set:
                self.write_set.append(operation.target)

    def init_target_local(self):
        """We want to make the local targets of Operations dependent on the Transaction.
        So we'll create a set of shared local targets in the Transaction."""
        self.all_targets = []
        # Récolter les targets
        for operation in self.operation_list:
            if operation.target not in self.all_targets:
                self.all_targets.append(operation.target)
        # Create copy of all_targets
        self.all_targets_copy = [copy(target) for target in self.all_targets]
        # Faire des pointeurs vers ces copies locales dans chaque operation
        for operation in self.operation_list:
            target_id = self.all_targets.index(operation.target)
            operation.target_local = self.all_targets_copy[target_id]

    def phase_read(self):
        """Apply the current operation.
        Each operation changes its local target."""
        success = self.operation_list[self.operation_iter].apply()
        if success:
            self.operation_iter += 1 
        # If we read all operations :
        if self.operation_iter >= len(self.operation_list):
            return True
        else:
            return False

    def phase_validate(self, other_transactions):
        earlier_transactions = []
        for transaction in other_transactions:
            if transaction.ts_start_read:
                if transaction.ts_start_read < self.ts_start_read:
                    earlier_transactions.append(transaction)
        for transaction in earlier_transactions:
            # Test 1
            if transaction.ts_finished == None:
                test_1 = False
            else:
                test_1 = transaction.ts_finished < self.ts_start_read
            # Test 3
            if transaction.ts_start_validate == None:
                test_3 = False
            else:
                test_3 = (
                    transaction.ts_start_validate < self.ts_start_validate 
                    and list_intersect(transaction.write_set, self.read_set) == [] 
                    and list_intersect(transaction.write_set, self.write_set) == []
                )
            # Test 2
            if transaction.ts_finished == None:
                test_2 = False
            else:
                # In the slides, it said to test "transaction.ts_finished < self.ts_start_write"
                # We never have started writing when we are in the validate phase.
                # So we replace "ts writing time" by the current timestamp 
                # That's also why we put test 2 last
                test_2 = transaction.ts_finished < time.time() and list_intersect(transaction.write_set, self.read_set) == []
            test = test_1 or test_2 or test_3
            if test is False:
                return False
        # We succeeded in every test. Return true. 
        return True
    

    def phase_write(self):
        """Commit changes of the current operation.
        Write the local target to the global target."""
        for i in range(len(self.all_targets)):
            self.all_targets = self.all_targets_copy[i]
        

    def apply_next(self, other_transactions) -> bool:
        if self.ts_finished:
            return True # Transaction has finished, skip
        # Otherwise, run transaction:
        # READ
        if self.phase == Transaction.phase_READ:
            success = self.phase_read()
            if success:
                self.phase = Transaction.phase_VALIDATE
                self.ts_start_validate = time.time() # Record time when validation begins 
        # VALIDATE
        elif self.phase == Transaction.phase_VALIDATE:
            success = self.phase_validate(other_transactions)
            if success:
                self.phase = Transaction.phase_WRITE
                self.ts_start_write = time.time()
            else:
                # Restart tranaction
                self.restart()
        # WRITE
        elif self.phase == Transaction.phase_WRITE:
            success = self.phase_write()
            if success:
                self.phase = Transaction.phase_FINISHED
                self.ts_finished = time.time()
        else:
            self.phase == Transaction.phase_READ
            self.ts_start_read = time.time()
        return success

    def restart(self):
        """Reset transaction"""
        # Current phase. None means that we didn't start. 
        self.phase = None
        # Timestamps of different phases
        self.ts_start_read = None
        self.ts_start_validate = None
        self.ts_start_write = None
        self.ts_finished = None
        # Important: create _new_ copies of the target objects. These were updated. 
        # Create local copies of target objects
        self.init_target_local()
        

In [33]:
from random import randint
from copy import copy
from typing import Callable, Iterator, Union, Optional, List

def run_transactions(max_iterations=10, *transactions: List[Transaction]):
    non_finished_transa = copy(list(transactions))
    curr_iteration = 0
    while len(non_finished_transa) > 0 and curr_iteration < max_iterations: 
        curr_iteration += 1
        n_transa = len(non_finished_transa)
        # Simulate random access to memory : we try to run the next step of a random transaction
        i = randint(0, n_transa-1) 
        curr_transa = non_finished_transa[i]
        curr_transa.apply_next()
        # Remove transactions from list if finished
        if curr_transa.finished:
            non_finished_transa.pop(i)



In [62]:
df = pd.DataFrame({"test": [1,2,3,55]})
test_target = Target(df)

def append_value(df: pd.DataFrame, value) -> pd.DataFrame:
    max_index = max(df.index) 
    df.loc[max_index + 1] = value 
    return df 

test_function = lambda df: append_value(df, 5)

test_transa_1 = Transaction("test_1", [
    Operation("append value", test_target, test_function),
    Operation("append second", test_target, test_function),
])

test_transa_2 = Transaction("test_2", [
    Operation("times 2", test_target, lambda x: x*2)
])

run_transactions(100, test_transa_1, test_transa_2)

test_1 Get lock of current operation append value
test_1 Apply current operation append value
test_1 Free lock of current operation append value
test_1 Get lock of current operation append second
test_1 Apply current operation append second
test_2 Get lock of current operation times 2
test_2 Get lock of current operation times 2
test_1 Free lock of current operation append second
test_2 Get lock of current operation times 2
test_2 Apply current operation times 2
test_2 Free lock of current operation times 2


In [63]:
test_target.df

,test
0,2
1,4
2,6
3,110
4,10
5,10
